In [ ]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CustomJS, FactorRange, Button
from bokeh.layouts import column
from bokeh.io import output_notebook

output_notebook()

# Sample monthly data
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]
monthly_values = [100, 150, 120, 130, 170, 140] #called monthly_sums

# Daily data for each month
daily_data = {
    "Jan": [str(i) for i in range(1, 32)], 
    "Feb": [str(i) for i in range(1, 29)],
    "Mar": [str(i) for i in range(1, 32)], 
    "Apr": [str(i) for i in range(1, 31)],
    "May": [str(i) for i in range(1, 32)], 
    "Jun": [str(i) for i in range(1, 31)]
}
daily_values = {
    "Jan": [10, 12, 9, 11, 13, 14, 15, 17, 16, 18, 19, 14, 13, 15, 16, 18, 19, 20, 21, 23, 22, 24, 26, 28, 27, 25, 24, 23, 21, 20, 19],
    "Feb": [8, 9, 7, 10, 11, 13, 14, 12, 15, 17, 16, 14, 13, 12, 11, 13, 15, 17, 16, 14, 13, 12, 10, 9, 7, 6, 8, 9],
    "Mar": [14, 15, 12, 11, 13, 12, 14, 15, 17, 16, 18, 19, 21, 20, 18, 17, 16, 15, 14, 16, 18, 20, 21, 23, 22, 24, 26, 28, 27, 25, 23],
    "Apr": [10, 11, 9, 8, 10, 9, 12, 14, 15, 17, 16, 18, 19, 21, 20, 18, 17, 16, 15, 14, 16, 18, 20, 21, 23, 22, 24, 26, 27, 25, 23],
    "May": [12, 13, 11, 10, 14, 15, 16, 17, 18, 19, 21, 20, 19, 18, 17, 16, 15, 14, 16, 18, 20, 21, 23, 22, 24, 26, 28, 27, 25, 23, 22],
    "Jun": [14, 15, 12, 11, 13, 12, 14, 15, 17, 16, 18, 19, 21, 20, 18, 17, 16, 15, 14, 16, 18, 20, 21, 23, 22, 24, 26, 28, 27, 25, 23]
}

# Initial data source
source = ColumnDataSource(data={"x": months, "y": monthly_values})

# Create figure
p = figure(x_range=FactorRange(*months), height=400, title="Monthly Data", tools="tap", toolbar_location=None)
p.vbar(x="x", top="y", width=0.5, source=source)

# Create a "Back to Monthly" Button (hidden initially)
button = Button(label="Back to Monthly Data", button_type="success", visible=False)

# JavaScript Callback for switching to daily data
callback = CustomJS(args=dict(source=source, p=p, button=button, daily_data=daily_data, daily_values=daily_values), code="""
    var selected = source.selected.indices[0]; //
    if (selected === undefined) return;

    var month = source.data['x'][selected];
    var days = daily_data[month];
    var values = daily_values[month];

    // Update source data to show daily values
    source.data = {x: days, y: values};  
    source.change.emit();

    // Update x-axis range dynamically
    p.x_range.factors = days;

    // Update title to show selected month
    p.title.text = "Daily Data of " + month;

    // Deselect bars to show all
    source.selected.indices = [];

    // Show back button
    button.visible = true;
""")

source.selected.js_on_change('indices', callback)

# JavaScript Callback for resetting to monthly view
reset_callback = CustomJS(args=dict(source=source, p=p, button=button), code="""
    var months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"];
    var monthly_values = [100, 150, 120, 130, 170, 140];

    // Restore original monthly data
    source.data = {x: months, y: monthly_values};  
    source.change.emit();

    // Reset x-axis range
    p.x_range.factors = months;

    // Reset title
    p.title.text = "Monthly Data";

    // Deselect any selected bars
    source.selected.indices = [];

    // Hide back button
    button.visible = false;
""")

button.js_on_click(reset_callback)

# Show plot and button
show(column(button, p))


Loading BokehJS ...

In [15]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Button
from bokeh.layouts import column
from bokeh.events import Tap

# Sample monthly data
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun"]
monthly_values = [100, 150, 120, 130, 170, 140]

# Daily data for each month
daily_data = {
    "Jan": [str(i) for i in range(1, 32)], 
    "Feb": [str(i) for i in range(1, 29)],
    "Mar": [str(i) for i in range(1, 32)], 
    "Apr": [str(i) for i in range(1, 31)],
    "May": [str(i) for i in range(1, 32)], 
    "Jun": [str(i) for i in range(1, 31)]
}
daily_values = {m: [i + 5 for i in range(len(daily_data[m]))] for m in months}

# Data sources
source_monthly = ColumnDataSource(data={"x": months, "y": monthly_values})
source_daily = ColumnDataSource(data={"x": [], "y": []})  # Empty initially

# Monthly figure
p_monthly = figure(x_range=months, height=400, title="Monthly Data", tools="tap")
p_monthly.vbar(x="x", top="y", width=0.5, source=source_monthly)

# Daily figure (empty initially)
p_daily = figure(x_range=[], height=400, title="Daily Data", tools="tap")
p_daily.vbar(x="x", top="y", width=0.5, source=source_daily)

# Back button (initially hidden)
button = Button(label="Back to Monthly Data", button_type="success", visible=False)

# Event for clicking on a monthly bar
def switch_to_daily(event):
    try:
        month = source_monthly.data["x"][event.indices[0]]
        source_daily.data = {"x": daily_data[month], "y": daily_values[month]}  # Update daily data
        p_daily.x_range.factors = daily_data[month]  # Update x-axis categories
        p_daily.title.text = f"Daily Data of {month}"  # Update title

        # Switch figures
        layout.children[0] = column(button, p_daily)
        button.visible = True
    except IndexError:
        pass  # Do nothing if no bar is clicked

p_monthly.on_event(Tap, switch_to_daily)

# Event for back button
def switch_to_monthly():
    layout.children[0] = column(button, p_monthly)  # Switch back to monthly figure
    button.visible = False

button.on_click(switch_to_monthly)

# Layout
layout = column(button, p_monthly)  # Show monthly view by default
show(layout)


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/js_callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

